# Explore here

In [2]:
import numpy as np, random
import pandas as pd
import dotenv
df_raw = pd.read_csv('C:/Users/Josue/4GA.DataScience/data/raw/Spam.csv')


ModuleNotFoundError: No module named 'numpy'

In [1]:
df_raw.info()
print('\n','---------------------------------------------------------------------------------------','\n',)
df_raw.shape

NameError: name 'df_raw' is not defined

In [ ]:
#Pasamos un filtro para duplicados, aunque parece que el archivo en cuanto a datos faltantes esta bien!
df_raw= df_raw.drop_duplicates()
df_raw=df_raw.reset_index(inplace= False, drop = True)
df_raw.info


<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2364  https://www.theverge.com/2020/6/29/21306889/di...    False
2365  https://www.smartcitiesworld.net/news/news/dee...    False
2366  https://techcrunch.com/2019/07/04/an-optimisti...    False
2367  https://www.technologyreview.com/2019/12/20/13...    False
2368       https://www.bbc.com/news/technology-51018758    False

[2369 rows x 2 columns]>

Entendemos que la segunda columna es la variable target, columna que indica si la primera es spam o no.
Como no hay un conjunto para usar y predecir mediante el modelo entrenado creamos el modelo y lo guardamos.

In [ ]:
#1 Transformamos los datos. La columna descriptiva tendremos que eliminar todo lo que no sean letras. Y sintaxis
#De la estructura de un vinculo o enlace web, como wwww/https/.com/es/info... etc
#La segunda columna la factorizamos , pasamos el valor binario booleano true , false a 0 o 1.
#Si en el paso 1 no lo hacemos correctamente el Modelo SVC 'Técnicamente hablando' tendria fuga de datos
#Ya que la estructura de una pagina web es muy reconocible y se facilitaria demasiado al modelo saber si es 
#Spam

In [ ]:
#Pasamos la columna Booleana a 1 o 0:
df_raw["is_spam"] = df_raw["is_spam"].apply(lambda x: 1 if x else 0).astype(int)
df_raw.head(6)

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1
5,https://www.brookings.edu/interactives/reopeni...,0


In [ ]:
import re
from urllib.parse import urlparse
import pandas as pd

def filtrotexto(text):
    text = re.sub(r'[^a-z ]', " ", text.lower())
    text = re.sub(r'\s+[a-z]\s+', " ", text)
    text = re.sub(r'^[a-z]\s+', " ", text)
    text = re.sub(r'\s+[a-z]$', " ", text)
    text = re.sub(r'\s+', " ", text).strip()
    return text.split()

def limpiarurl_mejorado(url):
    parsed_url = urlparse(url)
    cleaned_parts = []

    
    if parsed_url.scheme:
        cleaned_parts.extend(preprocess_url_part(parsed_url.scheme.replace('s', ''))) 
    if parsed_url.netloc:
        netloc_parts = parsed_url.netloc.split('.')
        for part in netloc_parts:
            sub_parts = part.split('-')
            for sub_part in sub_parts:
                cleaned_parts.extend(preprocess_url_part(sub_part))    
    if parsed_url.path:
        path_parts = parsed_url.path.split('/')
        for part in path_parts:
            cleaned_parts.extend(preprocess_url_part(part))
    return [part for part in cleaned_parts if part] # Remove empty strings
df_raw["url"] = df_raw["url"].apply(filtrotexto)
print(df_raw.head(6))

                                                 url  is_spam
0  [https, briefingday, us, list, manage, com, un...        1
1                           [https, www, hvper, com]        1
2                    [https, briefingday, com, v, i]        1
3          [https, briefingday, com, m, commentform]        0
4                     [https, briefingday, com, fan]        1
5  [https, www, brookings, edu, interactives, reo...        0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib
import os 

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_raw['url'].apply(lambda x: ' '.join(x)))
y = df_raw['is_spam']
model = SVC()
kf = KFold(n_splits=3, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kf)
print(f'Puntuaciones de validación cruzada: {scores}')
print(f'Puntuación media de validación cruzada: {scores.mean()}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
model_path = os.path.join('models', 'C:/Users/Josue/4GA.DataScience/models/')
vectorizer_path = os.path.join('models', 'tfidf_vectorizer.joblib')
os.makedirs('models', exist_ok=True) 
joblib.dump(model, model_path)
joblib.dump(vectorizer, vectorizer_path)

ModuleNotFoundError: No module named 'pandas'